In [9]:
# import datetime
# from orbit_predictor.sources import get_predictor_from_tle_lines
# import ephem
from sgp4.api import Satrec
# from math import sqrt
# import numpy as np

In [2]:
TLE_LINES = (
    "1 44713U 19074A   23001.33310695  .00000538  00000-0  55037-4 0  9996",
    "2 44713  53.0541   9.1431 0002180  39.3908 320.7239 15.06389052173400"
)
name = "STARLINK"


In [10]:
TLE_LINES = (
    "1 25544U 98067A   23001.26991367  .00015968  00000-0  28903-3 0  9993",
    "2 25544  51.6448  75.3411 0005204 209.5406 257.0907 15.49811030375904"
)
name = "ISS (ZARYA)"


In [11]:
line1 = TLE_LINES[0]
line2 = TLE_LINES[1]

In [6]:
predictor = get_predictor_from_tle_lines(TLE_LINES)
L = predictor.get_position(datetime.datetime(2023,1,1,17,32,55))  

In [7]:
predictor.get_position(datetime.datetime.now())

Position(when_utc=datetime.datetime(2023, 5, 24, 9, 56, 49, 987064), position_ecef=(2785.3038860385927, -3217.8235568895525, -5470.226357090315), velocity_ecef=(4.8175716122521415, 5.771944398524003, -0.9437102787403271), error_estimate=None)

In [13]:
tle_rec = ephem.readtle(name, line1, line2)
tle_rec.compute()


### SGP4 lib

In [12]:
satellite=Satrec.twoline2rv(line1, line2)

In [13]:
print(satellite.jdsatepoch)
print(satellite.jdsatepochF)


2459945.5
0.26991367


In [14]:
# print positions and velocities for 1-minute intervals
max_min = 10
for i in range(0, max_min):
    frac_min = i / (60 * 24) 
    e, r, v = satellite.sgp4(satellite.jdsatepoch, satellite.jdsatepochF + frac_min)
    # only 6 digits after the decimal in the output
    x = "%.6f" % r[0]
    y = "%.6f" % r[1]
    z = "%.6f" % r[2]
    vx = "%.6f" % v[0]
    vy = "%.6f" % v[1]
    vz = "%.6f" % v[2]
    print(i, x, y, z, vx, vy, vz)

0 -4398.682067 -854.238350 5100.905984 -0.544288 -7.448078 -1.717692
1 -4421.260325 -1298.829535 4986.230934 -0.208022 -7.365991 -2.103374
2 -4423.624429 -1737.482416 4848.693373 0.129262 -7.250206 -2.479487
3 -4405.759404 -2168.189818 4688.919352 0.466027 -7.101237 -2.844308
4 -4367.742571 -2588.979975 4507.636943 0.800734 -6.919752 -3.196160
5 -4309.743330 -2997.925543 4305.673063 1.131854 -6.706563 -3.533426
6 -4232.022528 -3393.152461 4083.949841 1.457869 -6.462633 -3.854552
7 -4134.931407 -3772.848614 3843.480536 1.777284 -6.189064 -4.158056
8 -4018.910139 -4135.272246 3585.365025 2.088631 -5.887098 -4.442532
9 -3884.485946 -4478.760085 3310.784855 2.390479 -5.558109 -4.706665


In [8]:
# print positions and velocities to a file
with open('../../data/teme/sgp4-teme.txt', 'w') as f:
    for i in range(0, max_min):
        frac_min = i / (60 * 24) 
        e, r, v = satellite.sgp4(satellite.jdsatepoch, satellite.jdsatepochF + frac_min)
        f.write(
            f"{satellite.jdsatepoch} {satellite.jdsatepochF + frac_min} {r[0]} {r[1]} {r[2]} {v[0]} {v[1]} {v[2]}\n")


### Skyfield lib

In [15]:
from skyfield.api import load, wgs84, EarthSatellite, Topos

In [16]:
ts = load.timescale()
satellite_sky = EarthSatellite(line1, line2, 'ISS', ts)


In [17]:
geocentric = satellite_sky.at(satellite_sky.epoch)
print(geocentric.position.km)
print(geocentric.velocity.km_per_s)


[-4391.71137577  -831.46977883  5110.66513642]
[-0.58639179 -7.445225   -1.71620452]


In [24]:
mine = np.array([6835086.828726, 1107352.760631, 9550.439149])

In [27]:
sum((geocentric.position.m - mine)**2)**0.5

49439.129265730095